<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a6edd02193bafc5018099285b3531e5837b0d5e05f558db7fdd0c74fd9da1a64
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-09 09:42:49
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: -271.00 (-0.0%)
Current PnL: -21.14 L (-14.03%)
CY Booked + Current PnL: -9.70 L (-6.44%)
-------------------
Total profit:  1.45 L
Total loss:  -22.59 L
-------------------
Total Booked + Current PnL: 17.28 L (13.97%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.65 L (63.12%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,71.0,H-MC,14.61,120893.0,20389.0,8716.0,2.00,20.29,7.21,28.96,79.0,2.34,0.86,61.14,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.38,39.0,M-SC,1.00,83303.0,-17474.0,17569.0,0.09,-17.34,21.09,0.09,245.0,-0.99,0.60,9.44,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.02,46.0,H-MC,3.02,188100.0,23430.0,19901.0,-0.60,14.23,10.58,26.31,89.0,1.18,1.34,23.35,X40N,NTT,AC
19,CIPLA,1662.74,-22.52,44.0,H-LC,3.66,206760.0,2260.0,21027.0,0.98,1.11,10.17,11.39,10.0,0.11,1.48,10.16,X40N,ATH,PHARMA
84,VOLTAS,1530.00,-1.78,50.0,H-MC,1.15,205995.0,14253.0,23504.0,0.44,7.43,11.41,19.69,99.0,0.61,1.47,14.74,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.00,108.87,64.0,H-SC,13.61,131868.0,-10611.0,30132.0,2.39,-7.45,22.85,13.70,163.0,-0.35,0.94,55.64,OX40N,NTT,BANKS
55,RAJOOENG,143.10,-41.44,42.0,H-SC,16.47,88840.0,-13660.0,54263.0,2.23,-13.33,61.08,39.61,114.0,-0.25,0.64,8.50,AR,ATH,MISC
50,NATIONALUM,244.55,-39.52,71.0,H-MC,14.61,120893.0,20389.0,8716.0,2.00,20.29,7.21,28.96,79.0,2.34,0.86,61.14,MH,ATH,METALS
30,HCLTECH,1937.36,1.51,60.0,H-LC,9.64,232046.0,-9878.0,72120.0,1.71,-4.08,31.08,25.73,8.0,-0.14,1.66,9.65,X40,ATH,IT
60,SAIL,228.00,45.95,56.0,M-MC,12.56,230541.0,5579.0,162070.0,1.64,2.48,70.30,74.53,192.0,0.03,1.65,35.53,XY24,BTT,STEEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,3748.65,56.0,M-SC,6.28,172924.0,-24208.0,114043.0,-2.15,-12.28,65.95,45.57,236.0,-0.21,1.24,22.60,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-34.58,45.0,M-SC,4.39,97865.0,-30805.0,76100.0,-1.82,-23.94,77.76,35.20,219.0,-0.40,0.70,12.33,X40,NTT,FOOTWEAR
82,VALIANTORG,838.00,-295.26,33.0,H-SC,1.72,108579.0,-55026.0,176343.0,-1.74,-33.63,162.41,74.15,139.0,-0.31,0.78,39.18,XR,NTT,CHEMICALS
52,PGHH,17907.65,-27.11,58.0,H-MC,2.05,210825.0,10005.0,57787.0,-1.63,4.98,27.41,33.76,80.0,0.17,1.51,10.44,X40,ATH,FMCG
58,REPCOHOME,880.00,-59.02,59.0,H-SC,2.75,236632.0,-49873.0,307219.0,-1.40,-17.41,129.83,89.82,134.0,-0.16,1.69,25.20,XY24,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.0,-13.79,49.0,M-LC,5.68,151875.0,930.0,108530.0,0.28,0.62,71.46,72.51,53.0,0.01,1.09,6.68,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,108.87,64.0,H-SC,13.61,131868.0,-10611.0,30132.0,2.39,-7.45,22.85,13.70,163.0,-0.35,0.94,55.64,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.38,39.0,M-SC,1.00,83303.0,-17474.0,17569.0,0.09,-17.34,21.09,0.09,245.0,-0.99,0.60,9.44,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,47.0,H-SC,1.97,220500.0,-49167.0,85510.0,0.16,-18.23,38.78,13.47,138.0,-0.57,1.58,12.12,XY24,NTT,PAINTS
18,CERA,9475.0,-22.91,40.0,H-SC,1.65,140656.0,-35247.0,77262.0,-1.00,-20.04,54.93,23.89,149.0,-0.46,1.01,21.48,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-23.04,50.0,H-MC,6.57,104139.0,-27696.0,69919.0,0.38,-21.01,67.14,32.03,98.0,-0.40,0.74,16.64,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,71.0,H-MC,14.61,120893.0,20389.0,8716.0,2.0,20.29,7.21,28.96,79.0,2.34,0.86,61.14,MH,ATH,METALS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.95,56.0,M-MC,12.56,230541.0,5579.0,162070.0,1.64,2.48,70.30,74.53,192.0,0.03,1.65,35.53,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,69.0,M-LC,7.19,215108.0,10032.0,102542.0,0.82,4.89,47.67,54.89,52.0,0.10,1.54,30.17,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.02,46.0,H-MC,3.02,188100.0,23430.0,19901.0,-0.60,14.23,10.58,26.31,89.0,1.18,1.34,23.35,X40N,NTT,AC
84,VOLTAS,1530.00,-1.78,50.0,H-MC,1.15,205995.0,14253.0,23504.0,0.44,7.43,11.41,19.69,99.0,0.61,1.47,14.74,XY25,NTT,AC
17,CAMS,5211.76,-6.94,42.0,H-SC,1.67,105843.0,3839.0,40083.0,0.00,3.76,37.87,43.06,122.0,0.10,0.76,21.52,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,71.0,H-MC,14.61,120893.0,20389.0,8716.0,2.00,20.29,7.21,28.96,79.0,2.34,0.86,61.14,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,64.0,H-MC,11.16,191989.0,12181.0,100237.0,-1.12,6.77,52.21,62.52,88.0,0.12,1.37,43.17,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,69.0,M-LC,7.19,215108.0,10032.0,102542.0,0.82,4.89,47.67,54.89,52.0,0.10,1.54,30.17,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.08,45.0,H-SC,8.95,128202.0,4866.0,126766.0,-0.77,3.95,98.88,106.72,119.0,0.04,0.92,28.25,AR,ATH,MISC
86,WIPRO,420.00,-13.79,49.0,M-LC,5.68,151875.0,930.0,108530.0,0.28,0.62,71.46,72.51,53.0,0.01,1.09,6.68,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.35,28.0,H-LC,10.41,276222.0,-39620.0,152281.0,-0.44,-12.54,55.13,35.67,5.0,-0.26,1.97,0.00,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.55,33.0,H-LC,5.64,205709.0,-46059.0,99275.0,0.39,-18.29,48.26,21.13,27.0,-0.46,1.47,10.65,X40,ATH,PAINTS
1,ABB,7934.00,-41.42,38.0,H-LC,7.54,244632.0,-16987.0,136211.0,-0.62,-6.49,55.68,45.57,7.0,-0.12,1.75,4.09,AR,NTT,ELECTRICAL
31,HINDUNILVR,2922.00,-10.67,40.0,H-LC,8.01,250740.0,-6323.0,41472.0,0.29,-2.46,16.54,13.67,24.0,-0.15,1.79,17.45,XY25,NTT,FMCG
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-0.68,51.0,H-LC,1.96,159640.0,-26455.0,73929.0,-0.75,-14.22,46.31,25.51,15.0,-0.36,1.14,17.64,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
48,LTIM,7230.2,-2.60,59.0,H-LC,2.52,241312.0,-4839.0,84049.0,0.37,-1.97,34.83,32.18,16.0,-0.06,1.73,33.37,X200,ATH,IT
51,NESTLEIND,1377.0,-10.51,50.0,H-LC,2.60,275465.0,10039.0,46746.0,-0.28,3.78,16.97,21.40,11.0,0.21,1.97,10.68,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.41,48.0,H-LC,3.09,214687.0,4321.0,24453.0,0.64,2.05,11.39,13.68,37.0,0.18,1.53,18.92,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.35,28.0,H-LC,10.41,276222.0,-39620.0,152281.0,-0.44,-12.54,55.13,35.67,5.0,-0.26,1.97,0.00,X40N,ATH,FMCG
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
1,ABB,7934.00,-41.42,38.0,H-LC,7.54,244632.0,-16987.0,136211.0,-0.62,-6.49,55.68,45.57,7.0,-0.12,1.75,4.09,AR,NTT,ELECTRICAL
80,UNITDSPR,1644.00,-12.80,49.0,H-LC,6.68,228554.0,-5590.0,54213.0,-0.32,-2.39,23.72,20.77,86.0,-0.10,1.63,4.37,X40N,NTT,BREWERIES
74,TCS,4406.34,-28.36,51.0,H-LC,12.18,284914.0,-61042.0,129294.0,0.13,-17.64,45.38,19.73,1.0,-0.47,2.04,4.94,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-0.68,51.0,H-LC,1.96,159640.0,-26455.0,73929.0,-0.75,-14.22,46.31,25.51,15.0,-0.36,1.14,17.64,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.55,33.0,H-LC,5.64,205709.0,-46059.0,99275.0,0.39,-18.29,48.26,21.13,27.0,-0.46,1.47,10.65,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.52,44.0,H-LC,3.66,206760.0,2260.0,21027.0,0.98,1.11,10.17,11.39,10.0,0.11,1.48,10.16,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-14.41,48.0,H-LC,3.09,214687.0,4321.0,24453.0,0.64,2.05,11.39,13.68,37.0,0.18,1.53,18.92,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,51.0,H-LC,12.18,284914.0,-61042.0,129294.0,0.13,-17.64,45.38,19.73,1.0,-0.47,2.04,4.94,X40,ATH,IT
40,INFY,2275.00,-17.80,55.0,H-LC,7.91,319670.0,6782.0,164918.0,0.39,2.17,51.59,54.87,3.0,0.04,2.29,8.96,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
83,VBL,671.64,-23.35,28.0,H-LC,10.41,276222.0,-39620.0,152281.0,-0.44,-12.54,55.13,35.67,5.0,-0.26,1.97,0.00,X40N,ATH,FMCG
1,ABB,7934.00,-41.42,38.0,H-LC,7.54,244632.0,-16987.0,136211.0,-0.62,-6.49,55.68,45.57,7.0,-0.12,1.75,4.09,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,42.0,L-SC,29.20,78336.0,-35213.0,75265.0,0.32,-31.01,96.08,35.27,268.0,-0.47,0.56,91.73,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,41.0,H-SC,10.63,90660.0,-7320.0,28921.0,-0.59,-7.47,31.90,22.05,152.0,-0.25,0.65,31.78,XR,ATH,FINANCE
50,NATIONALUM,244.55,-39.52,71.0,H-MC,14.61,120893.0,20389.0,8716.0,2.00,20.29,7.21,28.96,79.0,2.34,0.86,61.14,MH,ATH,METALS
78,UJJIVANSFB,60.00,108.87,64.0,H-SC,13.61,131868.0,-10611.0,30132.0,2.39,-7.45,22.85,13.70,163.0,-0.35,0.94,55.64,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-181.82,62.0,M-SC,14.26,142290.0,-6930.0,151112.0,-0.62,-4.64,106.20,96.62,208.0,-0.05,1.02,53.99,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-181.82,62.0,M-SC,14.26,142290.0,-6930.0,151112.0,-0.62,-4.64,106.20,96.62,208.0,-0.05,1.02,53.99,XY25,NTT,FINANCE
69,SURYODAY,240.00,62.28,62.0,H-SC,8.02,148951.0,-30120.0,95850.0,0.99,-16.82,64.35,36.71,135.0,-0.31,1.06,47.79,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,69.0,M-LC,7.19,215108.0,10032.0,102542.0,0.82,4.89,47.67,54.89,52.0,0.10,1.54,30.17,X5K,ATH,METALS
26,GREENPANEL,537.00,235.92,61.0,M-SC,4.93,149938.0,-33140.0,112124.0,0.00,-18.10,74.78,43.14,230.0,-0.30,1.07,41.04,XY24,NTT,MISC
50,NATIONALUM,244.55,-39.52,71.0,H-MC,14.61,120893.0,20389.0,8716.0,2.00,20.29,7.21,28.96,79.0,2.34,0.86,61.14,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.10
1,25,43.39
2,50,75.10


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.07
LC    32.23
MC    23.69
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.97
X40      14.67
X40N     11.15
XR       10.93
XY25     10.37
AR        8.17
OX40N     7.71
X200      1.73
MH        1.62
X5K       1.54
SR        1.13
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.00
H-LC    25.73
H-MC    20.40
M-SC    13.66
M-LC     5.49
M-MC     2.99
L-SC     1.41
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.69,-7.20,42.53
IT,12.90,-16.72,79.94
FINANCE,10.49,-14.96,70.47
MISC,6.94,-18.49,84.15
BANKS,6.32,-12.51,71.70
PAINTS,5.51,-19.84,37.70
ELECTRICAL,5.34,-10.25,48.93
INSURANCE,3.79,-4.22,43.18
AUTO,3.51,-17.25,73.30


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3241171.0,22
XR,1381985.0,14
AR,1188914.0,9
X40,789285.0,10
X40N,674234.0,9
OX40N,566666.0,10
XY25,485873.0,7
SR,263887.0,2
X5K,102542.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3736534.0,29
M-SC,1523242.0,17
H-MC,1297092.0,15
H-LC,1194785.0,15
M-LC,382058.0,4
M-MC,365112.0,2
L-SC,263325.0,3
L-MC,59968.0,1
L-LC,42573.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1209487.0      6
M-SC       XY24       846555.0      7
H-SC       AR         809409.0      5
           XR         806556.0      7
H-MC       XY24       579662.0      4
H-LC       X40        491035.0      5
M-MC       XY24       365112.0      2
H-SC       X40N       309822.0      4
           SR         263887.0      2
           OX40N      260006.0      4
H-LC       X40N       227521.0      3
H-MC       X40        222150.0      4
H-LC       AR         210140.0      2
H-MC       XY25       179517.0      2
L-SC       XR         174162.0      2
M-LC       XY24       170986.0      2
M-SC       AR         169365.0      2
           XY25       151112.0      1
           OX40N      147578.0      4
H-MC       X40N       136891.0      2
M-SC       XR         132532.0      2
H-LC       XY25       112671.0      3
M-LC       XR         108530.0      1
           X5K        102542.0      1
H-MC       XR         100237.0      1
L-SC       OX40N       89163.0      1
H-LC       X200        84049.0      1
H-SC       MH          77367.0      1
M-SC       X40         76100.0      1
H-MC       OX40N       69919.0      1
H-LC       XY24        69369.0      1
L-MC       XR          59968.0      1
L-LC       XY25        42573.0      1
H-MC       MH           8716.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
